In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
drive_path = "gdrive/MyDrive/MachineLearning/HandsOnMachineLearning/chapter17"

# 9.